# EDA in the first Raw Stage 

Do some Exploratory Data Anallysis in the first raw stage

## Initialize settigns and values

In [2]:
import pandas as pd
import os
import duckdb

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspaces/spotify_insights/airflow/credentials/google_credentials.json'
source_path = "gs://spotify-insights-pipeline-data/raw-tables"

In [4]:
raw_spotify_streams_all = pd.read_parquet(source_path, engine="pyarrow")
raw_spotify_streams_all


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,username,snapshot_date
0,2022-08-30T22:02:43Z,"Android OS 11 API 30 (Xiaomi, 21091116AG)",515866,ES,31.4.205.188,American Pie,Don McLean,The Best Of Don McLean,spotify:track:2QgWuCtBpNIpl5trmKCxRf,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10
1,2022-08-30T22:05:33Z,"Android OS 11 API 30 (Xiaomi, 21091116AG)",170132,ES,31.4.205.188,Peace Frog,The Doors,Morrison Hotel,spotify:track:5piJiL6aRhvrKBa9YuEfS9,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10
2,2022-08-30T22:09:59Z,"Android OS 11 API 30 (Xiaomi, 21091116AG)",265067,ES,31.4.205.188,Casey Jones - 2013 Remaster,Grateful Dead,Workingman's Dead,spotify:track:7LbfuQVct78YoghmoPtsQ8,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10
3,2022-08-30T22:12:30Z,"Android OS 11 API 30 (Xiaomi, 21091116AG)",150640,ES,31.4.205.188,White Rabbit,Jefferson Airplane,Surrealistic Pillow,spotify:track:4vpeKl0vMGdAXpZiQB2Dtd,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10
4,2022-08-30T22:17:23Z,"Android OS 11 API 30 (Xiaomi, 21091116AG)",292080,ES,31.4.205.188,Learning to Fly,Pink Floyd,A Momentary Lapse of Reason,spotify:track:3ieDOGcOqVxScs2VZBzidt,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190317,2016-01-06T01:39:51Z,"Android OS 5.1 API 22 (motorola, XT1097)",215120,PE,132.184.64.170,Love Buzz - Remastered,Nirvana,Bleach: Deluxe Edition,spotify:track:3FUsMXBxA4V7eUwQ7B0HQO,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10
190318,2016-01-06T01:45:35Z,"Android OS 5.1 API 22 (motorola, XT1097)",343400,PE,132.184.64.170,Times Of Trouble,Temple Of The Dog,Temple Of The Dog,spotify:track:0VdUQMiRn5pdHhghT9mbMt,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10
190319,2016-01-06T01:49:24Z,"Android OS 5.1 API 22 (motorola, XT1097)",228666,PE,132.184.64.170,Brown Sugar,The Rolling Stones,Sticky Fingers,spotify:track:18ECoCsbkFEfCBjlPOaYBY,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10
190320,2016-01-06T01:54:06Z,"Android OS 5.1 API 22 (motorola, XT1097)",281160,PE,132.184.64.170,Heart-Shaped Box,Nirvana,In Utero - 20th Anniversary Remaster,spotify:track:5CsjhePtc1FN7ecxqhzWDm,None,...,None,trackdone,trackdone,False,False,False,nan,False,joseph-higaki,2024-12-10


## Analyze for a sngle user

In [5]:

raw_spotify_streams = raw_spotify_streams_all[(raw_spotify_streams_all['username'] == 'joseph-higaki') & (raw_spotify_streams_all['snapshot_date'] == '2024-12-10')]

## Column Insights and Transformations

### Getting a string list of columns, for easier manipulation later

In [6]:

columns = raw_spotify_streams.columns.tolist()
print(columns)


['ts', 'platform', 'ms_played', 'conn_country', 'ip_addr', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name', 'episode_show_name', 'spotify_episode_uri', 'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline', 'offline_timestamp', 'incognito_mode', 'username', 'snapshot_date']


In [7]:
['ts', 'platform', 'ms_played', 'conn_country', 'ip_addr', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name', 'episode_show_name', 'spotify_episode_uri', 'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline', 'offline_timestamp', 'incognito_mode', 'username', 'snapshot_date']

['ts',
 'platform',
 'ms_played',
 'conn_country',
 'ip_addr',
 'master_metadata_track_name',
 'master_metadata_album_artist_name',
 'master_metadata_album_album_name',
 'spotify_track_uri',
 'episode_name',
 'episode_show_name',
 'spotify_episode_uri',
 'reason_start',
 'reason_end',
 'shuffle',
 'skipped',
 'offline',
 'offline_timestamp',
 'incognito_mode',
 'username',
 'snapshot_date']

### See a Generic describe of all columns

In [8]:
raw_spotify_streams.describe().transpose()

,count,unique,top,freq
ts,190322,87300,2019-02-01T05:42:06Z,428
platform,190322,56,android,42656
ms_played,190322,35070,0,8354
conn_country,190322,12,PE,87364
ip_addr,190322,3797,90.71.246.110,21070
master_metadata_track_name,190322,15326,None,1320
master_metadata_album_artist_name,190322,4840,Pearl Jam,5554
master_metadata_album_album_name,190322,8964,Sublime,2260
spotify_track_uri,190322,18400,None,1320
episode_name,190322,307,None,189002


## Initial Insights

| column | comments |
| --- | --- | 
| `ts` | is not unique. UTC 0 |
| `offline_timestamp` | looks float, but seems int |
| `snapshot_date` | needs to be set to UTC 0 for standardization |

In [9]:
raw_spotify_streams.dtypes

ts                                     object
platform                               object
ms_played                              object
conn_country                           object
ip_addr                                object
master_metadata_track_name             object
master_metadata_album_artist_name      object
master_metadata_album_album_name       object
spotify_track_uri                      object
episode_name                           object
episode_show_name                      object
spotify_episode_uri                    object
reason_start                           object
reason_end                             object
shuffle                                object
skipped                                object
offline                                object
offline_timestamp                      object
incognito_mode                         object
username                             category
snapshot_date                        category
dtype: object

### Added columns at initial upload: user_name

In [10]:
# Remove the categorical initial designation
raw_spotify_streams['username'] = raw_spotify_streams['username'].astype('str')
raw_spotify_streams['snapshot_date'] = pd.to_datetime(raw_spotify_streams['snapshot_date']).dt.tz_localize('UTC')

raw_spotify_streams.dtypes

ts                                                object
platform                                          object
ms_played                                         object
conn_country                                      object
ip_addr                                           object
master_metadata_track_name                        object
master_metadata_album_artist_name                 object
master_metadata_album_album_name                  object
spotify_track_uri                                 object
episode_name                                      object
episode_show_name                                 object
spotify_episode_uri                               object
reason_start                                      object
reason_end                                        object
shuffle                                           object
skipped                                           object
offline                                           object
offline_timestamp              

### Offline Timestamp 

Has float like values, but, most likely is Int

In [11]:
# Treat as float
raw_spotify_streams['offline_timestamp_float'] = raw_spotify_streams['offline_timestamp'].astype('float64')

#offline_timestamp
#top	nan
#freq	69563
# Check the number of missing (NaN) values in the column
missing_values = raw_spotify_streams['offline_timestamp_float'].isna().sum()
print(f"Number of missing values: {missing_values}")

# Round values
raw_spotify_streams['offline_timestamp_rounded'] = pd.to_numeric(raw_spotify_streams['offline_timestamp_float'], errors='coerce').round()
# Check it can be treated as int
raw_spotify_streams['offline_timestamp_float_rounded'] = raw_spotify_streams['offline_timestamp_float'] - raw_spotify_streams['offline_timestamp_rounded']

rounding_diff__values = raw_spotify_streams['offline_timestamp_float_rounded'].sum()
print(f"Total of rounding diff: {rounding_diff__values}")
# Sum shouldl be zero 

raw_spotify_streams['offline_timestamp_rounded'] = raw_spotify_streams['offline_timestamp_rounded'].astype('Int64')
# Remove original  columns 
raw_spotify_streams.drop(columns=['offline_timestamp', 'offline_timestamp_float', 'offline_timestamp_float_rounded'], inplace=True)

# promote int as official column
raw_spotify_streams.rename(columns={'offline_timestamp_rounded': 'offline_timestamp'}, inplace=True)


Number of missing values: 139126
Total of rounding diff: 0.0


## Set possible data types

In [12]:
# Define the desired data types
dtype_mapping = {
    'ts': 'datetime64[ns, UTC]',  # timestamp
    'platform': 'str',    # str
    'ms_played': 'Int64',    # integer with NaN support (nullable integer type)
    'conn_country': 'str',  # str
    'ip_addr': 'str',  # str
    'master_metadata_track_name': 'str',  # str
    'master_metadata_album_artist_name': 'str',  # str
    'master_metadata_album_album_name': 'str',  # str
    'spotify_track_uri': 'str',  # str
    'episode_name': 'str',  # str
    'episode_show_name': 'str',  # str
    'spotify_episode_uri': 'str',  # str
    'reason_start': 'str',  # str
    'reason_end': 'str',  # str
    'shuffle': 'bool',  # boolean
    'skipped': 'bool',  # boolean
    'offline': 'bool',  # boolean
    'offline_timestamp': 'Int64',  # nullable integer
    'incognito_mode': 'bool',  # boolean
    'username': 'str',  # str
    'snapshot_date': 'datetime64[ns, UTC]'  # date
}
col_dtypes = pd.DataFrame({
    'column_name': list(dtype_mapping.keys()),
    'dtype': list(dtype_mapping.values())
})

In [13]:
for dtype, group in col_dtypes.groupby('dtype'):
    print(dtype)
    display(list(group['column_name']))

Int64


['ms_played', 'offline_timestamp']

bool


['shuffle', 'skipped', 'offline', 'incognito_mode']

datetime64[ns, UTC]


['ts', 'snapshot_date']

str


['platform',
 'conn_country',
 'ip_addr',
 'master_metadata_track_name',
 'master_metadata_album_artist_name',
 'master_metadata_album_album_name',
 'spotify_track_uri',
 'episode_name',
 'episode_show_name',
 'spotify_episode_uri',
 'reason_start',
 'reason_end',
 'username']

## Convert to Data Type

In [14]:
# Apply conversions
for column, dtype in dtype_mapping.items():
    raw_spotify_streams[column] = raw_spotify_streams[column].astype(dtype)

#### Summary of transformation

In [15]:
def transform_dtypes(df):
    df['username'] = df['username'].astype('str')
    #df['snapshot_date'] = df['snapshot_date'].astype('str')    
    #df['snapshot_date'] = pd.to_datetime(df['snapshot_date']).dt.tz_localize('UTC')
    df['offline_timestamp_float'] = df['offline_timestamp'].astype('float64')
    df['offline_timestamp_rounded'] = pd.to_numeric(df['offline_timestamp_float'], errors='coerce').round().astype('Int64')
    # Remove original  columns 
    df.drop(columns=['offline_timestamp', 'offline_timestamp_float'], inplace=True)
    # promote int as official column
    df.rename(columns={'offline_timestamp_rounded': 'offline_timestamp'}, inplace=True)
    # Apply conversions
    for column, dtype in dtype_mapping.items():
        df[column] = df[column].astype(dtype)


In [16]:
Int64_columns = ['ms_played', 'offline_timestamp']
bool_columns = ['shuffle', 'skipped', 'offline', 'incognito_mode']
datetime64_columns = ['ts', 'snapshot_date']
str_columns = ['platform',
    'conn_country',
    'ip_addr',
    'master_metadata_track_name',
    'master_metadata_album_artist_name',
    'master_metadata_album_album_name',
    'spotify_track_uri',
    'episode_name',
    'episode_show_name',
    'spotify_episode_uri',
    'reason_start',
    'reason_end',
    'username']

## Describe each dtype

In [17]:
raw_spotify_streams[Int64_columns].describe()

,ms_played,offline_timestamp
count,190322.0,51196.0
mean,191944.464014,206851002097.027344
std,218083.901389,523230418848.982361
min,0.0,0.0
25%,57437.0,1680650266.0
50%,206380.0,1704617866.5
75%,264573.0,1721488753.0
max,13409663.0,1665465084392.0


In [18]:
raw_spotify_streams[datetime64_columns].describe()

,ts,snapshot_date
count,190322,190322
mean,2019-10-16 22:39:36.132837888+00:00,2024-12-10 00:00:00+00:00
min,2014-12-23 00:37:58+00:00,2024-12-10 00:00:00+00:00
25%,2016-04-30 16:58:58+00:00,2024-12-10 00:00:00+00:00
50%,2020-09-05 10:37:51+00:00,2024-12-10 00:00:00+00:00
75%,2022-08-09 04:53:55+00:00,2024-12-10 00:00:00+00:00
max,2024-12-05 15:25:10+00:00,2024-12-10 00:00:00+00:00


In [19]:
for column in bool_columns:
# Check the count of True and False values in the 'shuffle' column
    values = raw_spotify_streams[column].value_counts()
    display(values)


shuffle
True    190322
Name: count, dtype: int64

skipped
True    190322
Name: count, dtype: int64

offline
True    190322
Name: count, dtype: int64

incognito_mode
True    190322
Name: count, dtype: int64

In [20]:
raw_spotify_streams[str_columns].describe()

,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,username
count,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322
unique,56,12,3797,15326,4840,8964,18400,307,54,307,13,15,1
top,android,PE,90.71.246.110,None,Pearl Jam,Sublime,None,None,None,None,trackdone,trackdone,joseph-higaki
freq,42656,87364,21070,1320,5554,2260,1320,189002,189002,189002,120574,121112,190322


## Review None values at `master_metadata_track_name`  

In [21]:
# Filter for rows where 'master_metadata_track_name' is empty or None
filtered_data = raw_spotify_streams[raw_spotify_streams['master_metadata_track_name'].isna() | (raw_spotify_streams['master_metadata_track_name'] == '') | (raw_spotify_streams['master_metadata_track_name'] == 'None')]

# Display the filtered data
display(filtered_data.head(5))


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,username,snapshot_date,offline_timestamp
256,2022-09-10 20:17:31+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",404414,ES,47.60.39.194,None,None,None,None,America's Out-Of-Control Overdraft Fees - If Y...,...,spotify:episode:5Dyfh1Lg0Bvi2gtyfwBiJQ,clickrow,endplay,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>
257,2022-09-10 21:54:42+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",2308498,ES,31.4.206.171,None,None,None,None,This Week's Top Stories - The Royal Family ann...,...,spotify:episode:1KXpGlnOe9jnwsBzRMjZMJ,appload,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>
258,2022-09-10 21:54:42+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",50110,ES,31.4.206.171,None,None,None,None,This Week's Top Stories - The Royal Family ann...,...,spotify:episode:1KXpGlnOe9jnwsBzRMjZMJ,clickrow,logout,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>
259,2022-09-11 16:00:19+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",3033217,ES,90.161.22.203,None,None,None,None,The Road to Legalizing Cannabis at the Federal...,...,spotify:episode:1ocOstVxDCqT3rDD8MZx49,trackdone,logout,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1662844194504
284,2022-09-12 21:21:44+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",411167,ES,31.4.203.67,None,None,None,None,New York Fashion Week: Why Do We Celebrate This?,...,spotify:episode:07bt6swCrraox067YqhRaE,clickrow,endplay,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>


## Initial Insights

| column | comments |
| --- | --- | 
| `ts` | is not unique. UTC 0 |
| `offline_timestamp` | float input, rounded |
| `snapshot_date` | Date. Set to UTC 0 |
| `master_metadata_track_name` | Has String= 'None' values  |
| `master_metadata_album_artist_name` | Has String= 'None' values  |
| `master_metadata_album_album_name` | Has String= 'None' values  |
| `episode_name` | Has String= 'None' values  |
| `spotify_episode_uri` | Has String= 'None' values  |


## Some Categorical columns

In [22]:
see_cat_cols = [
    "conn_country",
    "reason_start",
    "reason_end"
]

for col in see_cat_cols:
    categories = raw_spotify_streams.groupby(col).size()
    df_category = categories.reset_index()  # Reset index to make it a DataFrame
    df_category.columns = [col, 'count']  # Rename columns to appropriate names
    df_category_sorted = df_category.sort_values(by='count', ascending=False)  # Sort by count in descending order
    display(df_category_sorted)


,conn_country,count
7,PE,87364
4,ES,85088
10,US,10662
0,AD,3338
11,ZZ,1812
6,NL,1426
1,AT,204
3,CL,192
5,FR,140
8,PT,50


,reason_start,count
10,trackdone,120574
5,fwdbtn,29134
3,clickrow,25800
1,appload,6560
2,backbtn,3872
7,playbtn,1678
8,remote,1214
12,unknown,802
0,,512
11,trackerror,164


,reason_end,count
10,trackdone,121112
6,fwdbtn,29098
5,endplay,22906
7,logout,5560
2,backbtn,3818
13,unexpected-exit-while-paused,3660
14,unknown,2378
8,remote,582
12,unexpected-exit,434
0,,418


## Analyse duplicates at `ts`

In [23]:
con = duckdb.connect()

# Register the DataFrame as a DuckDB table
con.register('raw_spotify_streams', raw_spotify_streams)

In [24]:
query = """
  with with_rn as (
    select 
    row_number() over (window_by_ts) as rn,
    count() over (window_by_ts) as count_by_ts,
    *
    from raw_spotify_streams
    window window_by_ts as (PARTITION BY ts)
  )
  select * 
  from with_rn 
  where count_by_ts > 1
  order by ts, rn
"""
duplicate_ts = con.execute(query).fetchdf()
transform_dtypes(duplicate_ts)
display(duplicate_ts)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,rn,count_by_ts,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,username,snapshot_date,offline_timestamp
0,1,2,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,None,appload,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>
1,2,2,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,None,appload,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>
2,1,2,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,None,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>
3,2,2,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,None,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>
4,1,2,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,...,None,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190317,2,2,2024-12-04 07:04:05+00:00,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,...,None,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1733295539
190318,1,2,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,None,trackdone,unknown,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1733295845
190319,2,2,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,None,trackdone,unknown,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1733295845
190320,1,2,2024-12-05 15:25:10+00:00,android,408,AD,85.94.179.153,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,None,appload,logout,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1733410872


In [25]:
def describe(df):
    display(df[Int64_columns].describe())
    display(df[datetime64_columns].describe())    
    display(df[str_columns].describe())
    for column in bool_columns:
    # Check the count of True and False values in the 'shuffle' column
        values = df[column].value_counts()
        display(values)


describe(duplicate_ts)

,ms_played,offline_timestamp
count,190322.0,51196.0
mean,191944.464014,206851002097.027283
std,218083.901389,523230418848.983032
min,0.0,0.0
25%,57437.0,1680650266.0
50%,206380.0,1704617866.5
75%,264573.0,1721488753.0
max,13409663.0,1665465084392.0


,ts,snapshot_date
count,190322,190322
mean,2019-10-16 22:39:36.132838144+00:00,2024-12-10 00:00:00+00:00
min,2014-12-23 00:37:58+00:00,2024-12-10 00:00:00+00:00
25%,2016-04-30 16:58:58+00:00,2024-12-10 00:00:00+00:00
50%,2020-09-05 10:37:51+00:00,2024-12-10 00:00:00+00:00
75%,2022-08-09 04:53:55+00:00,2024-12-10 00:00:00+00:00
max,2024-12-05 15:25:10+00:00,2024-12-10 00:00:00+00:00


,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,username
count,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322
unique,56,12,3797,15326,4840,8964,18400,307,54,307,13,15,1
top,android,PE,90.71.246.110,None,Pearl Jam,Sublime,None,None,None,None,trackdone,trackdone,joseph-higaki
freq,42656,87364,21070,1320,5554,2260,1320,189002,189002,189002,120574,121112,190322


shuffle
True    190322
Name: count, dtype: int64

skipped
True    190322
Name: count, dtype: int64

offline
True    190322
Name: count, dtype: int64

incognito_mode
True    190322
Name: count, dtype: int64

## Create a surrogate key

In [26]:
raw_spotify_streams_for_dedup = raw_spotify_streams.copy()

In [27]:
source_info_columns = ['ts', 'platform', 'ms_played', 'conn_country', 'ip_addr', 
                       'master_metadata_track_name', 'master_metadata_album_artist_name', 
                       'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name', 
                       'episode_show_name', 'spotify_episode_uri', 'reason_start', 'reason_end', 
                       'shuffle', 'skipped', 'offline', 'offline_timestamp', 'incognito_mode', 'username'
                       ]

## did not include snapshot date

In [28]:
import hashlib

def genMd5Hash(row, info_columns) -> str:
    string = '-'.join([str(row[col]) for col in info_columns])
    """ Function to generate md5 when given a string """
    return hashlib.md5(string.encode()).hexdigest()

In [29]:
def generate_surrogate_key(df):
    df['stream_id'] = df.apply(lambda row: genMd5Hash(row, source_info_columns),axis=1)

In [30]:
generate_surrogate_key(raw_spotify_streams_for_dedup)

In [31]:
display(raw_spotify_streams_for_dedup)

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,username,snapshot_date,offline_timestamp,stream_id
0,2022-08-30 22:02:43+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",515866,ES,31.4.205.188,American Pie,Don McLean,The Best Of Don McLean,spotify:track:2QgWuCtBpNIpl5trmKCxRf,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,07e9ca970c797d10a279c4dcd176c39d
1,2022-08-30 22:05:33+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",170132,ES,31.4.205.188,Peace Frog,The Doors,Morrison Hotel,spotify:track:5piJiL6aRhvrKBa9YuEfS9,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,2f36bb3360c8ad3fced43d3f59bc098b
2,2022-08-30 22:09:59+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",265067,ES,31.4.205.188,Casey Jones - 2013 Remaster,Grateful Dead,Workingman's Dead,spotify:track:7LbfuQVct78YoghmoPtsQ8,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,c85e1a213a843facd9950bb4fcd32aa9
3,2022-08-30 22:12:30+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",150640,ES,31.4.205.188,White Rabbit,Jefferson Airplane,Surrealistic Pillow,spotify:track:4vpeKl0vMGdAXpZiQB2Dtd,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,859b91c6798b68b68dde23ad611caa46
4,2022-08-30 22:17:23+00:00,"Android OS 11 API 30 (Xiaomi, 21091116AG)",292080,ES,31.4.205.188,Learning to Fly,Pink Floyd,A Momentary Lapse of Reason,spotify:track:3ieDOGcOqVxScs2VZBzidt,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,f91a64fb0c44ec4d87d1713c594ead60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190317,2016-01-06 01:39:51+00:00,"Android OS 5.1 API 22 (motorola, XT1097)",215120,PE,132.184.64.170,Love Buzz - Remastered,Nirvana,Bleach: Deluxe Edition,spotify:track:3FUsMXBxA4V7eUwQ7B0HQO,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,862953fb14aed9c5bb015a96c5e7047d
190318,2016-01-06 01:45:35+00:00,"Android OS 5.1 API 22 (motorola, XT1097)",343400,PE,132.184.64.170,Times Of Trouble,Temple Of The Dog,Temple Of The Dog,spotify:track:0VdUQMiRn5pdHhghT9mbMt,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,c62648afe3c4a7eead2fcb17a23416b2
190319,2016-01-06 01:49:24+00:00,"Android OS 5.1 API 22 (motorola, XT1097)",228666,PE,132.184.64.170,Brown Sugar,The Rolling Stones,Sticky Fingers,spotify:track:18ECoCsbkFEfCBjlPOaYBY,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,bdfdf25c25d9b2665862b84b4bd64bd2
190320,2016-01-06 01:54:06+00:00,"Android OS 5.1 API 22 (motorola, XT1097)",281160,PE,132.184.64.170,Heart-Shaped Box,Nirvana,In Utero - 20th Anniversary Remaster,spotify:track:5CsjhePtc1FN7ecxqhzWDm,None,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,<NA>,ff1da8cc6e6dae17a56593310d3081d6


In [32]:
# Register the DataFrame as a DuckDB table
con.register('raw_spotify_streams_for_dedup', raw_spotify_streams_for_dedup)

In [33]:
query = """
  with with_rn as (
    select 
    row_number() over (window_by_stream_id) as rn_by_stream_id,
    count() over (window_by_stream_id) as count_by_stream_id,
    *
    from raw_spotify_streams_for_dedup
    window window_by_stream_id as (PARTITION BY stream_id)
  )
  select * 
  from with_rn 
  where count_by_stream_id > 1
  order by ts, rn_by_stream_id
"""
duplicate_hash = con.execute(query).fetchdf()
duplicate_hash.dtypes


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

rn_by_stream_id                                        int64
count_by_stream_id                                     int64
ts                                   datetime64[us, Etc/UTC]
platform                                              object
ms_played                                              int64
conn_country                                          object
ip_addr                                               object
master_metadata_track_name                            object
master_metadata_album_artist_name                     object
master_metadata_album_album_name                      object
spotify_track_uri                                     object
episode_name                                          object
episode_show_name                                     object
spotify_episode_uri                                   object
reason_start                                          object
reason_end                                            object
shuffle                 

In [34]:
display(duplicate_hash)

,rn_by_stream_id,count_by_stream_id,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,username,snapshot_date,offline_timestamp,stream_id
0,1,2,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,appload,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d6b75fe3cc818d5b5d247228da525414
1,2,2,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,appload,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d6b75fe3cc818d5b5d247228da525414
2,1,2,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,baa6e5eea30ec3376c952bd60533c06e
3,2,2,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,baa6e5eea30ec3376c952bd60533c06e
4,1,2,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,26bc6434510998232ebaa64f2494f9d6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190317,2,2,2024-12-04 07:04:05+00:00,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d7f7990841af62ed57b41ce914936d96
190318,1,2,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,trackdone,unknown,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d83f376f1e232ec174353f6786a51bd4
190319,2,2,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,trackdone,unknown,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d83f376f1e232ec174353f6786a51bd4
190320,1,2,2024-12-05 15:25:10+00:00,android,408,AD,85.94.179.153,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,appload,logout,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733411e+09,a8bb14a49334d861e280cca6bab7fc21


In [35]:
describe(duplicate_hash)

,ms_played,offline_timestamp
count,1.903220e+05,5.119600e+04
mean,1.919445e+05,2.068510e+11
std,2.180839e+05,5.232304e+11
min,0.000000e+00,0.000000e+00
25%,5.743700e+04,1.680650e+09
50%,2.063800e+05,1.704618e+09
75%,2.645730e+05,1.721489e+09
max,1.340966e+07,1.665465e+12


,ts,snapshot_date
count,190322,190322
mean,2019-10-16 22:39:36.132838+00:00,2024-12-10 00:00:00+00:00
min,2014-12-23 00:37:58+00:00,2024-12-10 00:00:00+00:00
25%,2016-04-30 16:58:58+00:00,2024-12-10 00:00:00+00:00
50%,2020-09-05 10:37:51+00:00,2024-12-10 00:00:00+00:00
75%,2022-08-09 04:53:55+00:00,2024-12-10 00:00:00+00:00
max,2024-12-05 15:25:10+00:00,2024-12-10 00:00:00+00:00


,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,username
count,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322,190322
unique,56,12,3797,15326,4840,8964,18400,307,54,307,13,15,1
top,android,PE,90.71.246.110,None,Pearl Jam,Sublime,None,None,None,None,trackdone,trackdone,joseph-higaki
freq,42656,87364,21070,1320,5554,2260,1320,189002,189002,189002,120574,121112,190322


shuffle
True    190322
Name: count, dtype: int64

skipped
True    190322
Name: count, dtype: int64

offline
True    190322
Name: count, dtype: int64

incognito_mode
True    190322
Name: count, dtype: int64

## Duplicates
There are 910 rows that have identical row data 
We will only keep one 

In [36]:
query = """
  with with_rn as (
    select 
    row_number() over (window_by_stream_id) as rn_by_stream_id,
    count() over (window_by_stream_id) as count_by_stream_id,
    *
    from raw_spotify_streams_for_dedup
    window window_by_stream_id as (PARTITION BY stream_id)
  )
  select * 
  from with_rn 
  where rn_by_stream_id > 1
  order by ts, rn_by_stream_id
"""
duplicated_rows_hash = con.execute(query).fetchdf()
duplicated_rows_hash


,rn_by_stream_id,count_by_stream_id,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,username,snapshot_date,offline_timestamp,stream_id
0,2,2,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,appload,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d6b75fe3cc818d5b5d247228da525414
1,2,2,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,baa6e5eea30ec3376c952bd60533c06e
2,2,2,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,26bc6434510998232ebaa64f2494f9d6
3,2,2,2014-12-23 00:49:35+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",170800,US,66.165.162.68,Rock And Roll All Nite,KISS,Greatest Kiss,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,b1acc11f3a473570ee358f7d5256222a
4,2,2,2014-12-23 00:54:26+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",290280,US,66.165.162.68,Message In A Bottle - Remastered 2003,The Police,Reggatta De Blanc,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d65acd45c2786b387df0ebfebe340eb1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95645,2,2,2024-12-04 06:55:39+00:00,android,282653,ES,79.153.169.248,Voodoo,Godsmack,Godsmack,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733295e+09,1db47bdf6a952082b0d013e44417908c
95646,2,2,2024-12-04 06:58:59+00:00,android,199906,ES,79.153.169.248,Last Resort,Papa Roach,Infest,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733295e+09,a9efd464082ecb18990ff3c4143ecde6
95647,2,2,2024-12-04 07:04:05+00:00,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d7f7990841af62ed57b41ce914936d96
95648,2,2,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,trackdone,unknown,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d83f376f1e232ec174353f6786a51bd4


## Dedup by hash

In [62]:
query = """
  with with_rn as (
    select 
    row_number() over (window_by_stream_id) as rn_by_stream_id,
    count() over (window_by_stream_id) as count_by_stream_id,
    *
    from raw_spotify_streams_for_dedup
    window window_by_stream_id as (PARTITION BY stream_id)
  )
  select * 
  from with_rn 
  where rn_by_stream_id = 1
  order by ts, rn_by_stream_id
"""
dedup_by_hash = con.execute(query).fetchdf()
dedup_by_hash


,rn_by_stream_id,count_by_stream_id,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,username,snapshot_date,offline_timestamp,stream_id
0,1,2,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,appload,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d6b75fe3cc818d5b5d247228da525414
1,1,2,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,baa6e5eea30ec3376c952bd60533c06e
2,1,2,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,26bc6434510998232ebaa64f2494f9d6
3,1,2,2014-12-23 00:49:35+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",170800,US,66.165.162.68,Rock And Roll All Nite,KISS,Greatest Kiss,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,b1acc11f3a473570ee358f7d5256222a
4,1,2,2014-12-23 00:54:26+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",290280,US,66.165.162.68,Message In A Bottle - Remastered 2003,The Police,Reggatta De Blanc,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d65acd45c2786b387df0ebfebe340eb1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94667,1,2,2024-12-04 06:55:39+00:00,android,282653,ES,79.153.169.248,Voodoo,Godsmack,Godsmack,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733295e+09,1db47bdf6a952082b0d013e44417908c
94668,1,2,2024-12-04 06:58:59+00:00,android,199906,ES,79.153.169.248,Last Resort,Papa Roach,Infest,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733295e+09,a9efd464082ecb18990ff3c4143ecde6
94669,1,2,2024-12-04 07:04:05+00:00,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d7f7990841af62ed57b41ce914936d96
94670,1,2,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,trackdone,unknown,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d83f376f1e232ec174353f6786a51bd4


## Partition by ts year

In [38]:
dedup_by_hash.head(4)

,rn_by_stream_id,count_by_stream_id,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,username,snapshot_date,offline_timestamp,stream_id
0,1,2,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,appload,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d6b75fe3cc818d5b5d247228da525414
1,1,2,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,baa6e5eea30ec3376c952bd60533c06e
2,1,2,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,26bc6434510998232ebaa64f2494f9d6
3,1,2,2014-12-23 00:49:35+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",170800,US,66.165.162.68,Rock And Roll All Nite,KISS,Greatest Kiss,...,trackdone,trackdone,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,b1acc11f3a473570ee358f7d5256222a


In [63]:
dedup_by_hash['streamed_at'] = dedup_by_hash['ts']
dedup_by_hash['streamed_year'] = dedup_by_hash['streamed_at'].dt.year
dedup_by_hash


,rn_by_stream_id,count_by_stream_id,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,shuffle,skipped,offline,incognito_mode,username,snapshot_date,offline_timestamp,stream_id,streamed_at,streamed_year
0,1,2,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d6b75fe3cc818d5b5d247228da525414,2014-12-23 00:37:58+00:00,2014
1,1,2,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,baa6e5eea30ec3376c952bd60533c06e,2014-12-23 00:40:47+00:00,2014
2,1,2,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,26bc6434510998232ebaa64f2494f9d6,2014-12-23 00:46:44+00:00,2014
3,1,2,2014-12-23 00:49:35+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",170800,US,66.165.162.68,Rock And Roll All Nite,KISS,Greatest Kiss,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,b1acc11f3a473570ee358f7d5256222a,2014-12-23 00:49:35+00:00,2014
4,1,2,2014-12-23 00:54:26+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",290280,US,66.165.162.68,Message In A Bottle - Remastered 2003,The Police,Reggatta De Blanc,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,NaN,d65acd45c2786b387df0ebfebe340eb1,2014-12-23 00:54:26+00:00,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94667,1,2,2024-12-04 06:55:39+00:00,android,282653,ES,79.153.169.248,Voodoo,Godsmack,Godsmack,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733295e+09,1db47bdf6a952082b0d013e44417908c,2024-12-04 06:55:39+00:00,2024
94668,1,2,2024-12-04 06:58:59+00:00,android,199906,ES,79.153.169.248,Last Resort,Papa Roach,Infest,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733295e+09,a9efd464082ecb18990ff3c4143ecde6,2024-12-04 06:58:59+00:00,2024
94669,1,2,2024-12-04 07:04:05+00:00,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d7f7990841af62ed57b41ce914936d96,2024-12-04 07:04:05+00:00,2024
94670,1,2,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,True,True,True,True,joseph-higaki,2024-12-10 00:00:00+00:00,1.733296e+09,d83f376f1e232ec174353f6786a51bd4,2024-12-04 07:26:58+00:00,2024


In [65]:
new_columns = ['stream_id', 'streamed_at', 'streamed_year' ]
remove_columns=["rn_by_stream_id", "count_by_stream_id", "snapshot_date", "ts"]
old_columns =  [col for col in dedup_by_hash.columns if col not in (new_columns + remove_columns)]
dedup_by_hash = dedup_by_hash[new_columns + old_columns]

In [64]:
new_columns + remove_columns

['stream_id',
 'streamed_at',
 'streamed_year',
 'rn_by_stream_id',
 'count_by_stream_id',
 'snapshot_date',
 'ts']

In [58]:
[col for col in dedup_by_hash.columns if col not in [new_columns + remove_columns]]

['stream_id',
 'streamed_at',
 'streamed_year',
 'rn_by_stream_id',
 'count_by_stream_id',
 'ts',
 'platform',
 'ms_played',
 'conn_country',
 'ip_addr',
 'master_metadata_track_name',
 'master_metadata_album_artist_name',
 'master_metadata_album_album_name',
 'spotify_track_uri',
 'episode_name',
 'episode_show_name',
 'spotify_episode_uri',
 'reason_start',
 'reason_end',
 'shuffle',
 'skipped',
 'offline',
 'incognito_mode',
 'username',
 'snapshot_date',
 'offline_timestamp',
 'stream_id',
 'streamed_at',
 'streamed_year']

In [66]:
dedup_by_hash

,stream_id,streamed_at,streamed_year,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,username,offline_timestamp
0,d6b75fe3cc818d5b5d247228da525414,2014-12-23 00:37:58+00:00,2014,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,...,None,None,appload,trackdone,True,True,True,True,joseph-higaki,NaN
1,baa6e5eea30ec3376c952bd60533c06e,2014-12-23 00:40:47+00:00,2014,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,...,None,None,trackdone,trackdone,True,True,True,True,joseph-higaki,NaN
2,26bc6434510998232ebaa64f2494f9d6,2014-12-23 00:46:44+00:00,2014,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,...,None,None,trackdone,trackdone,True,True,True,True,joseph-higaki,NaN
3,b1acc11f3a473570ee358f7d5256222a,2014-12-23 00:49:35+00:00,2014,"Android OS 4.4.4 API 19 (motorola, XT1097)",170800,US,66.165.162.68,Rock And Roll All Nite,KISS,Greatest Kiss,...,None,None,trackdone,trackdone,True,True,True,True,joseph-higaki,NaN
4,d65acd45c2786b387df0ebfebe340eb1,2014-12-23 00:54:26+00:00,2014,"Android OS 4.4.4 API 19 (motorola, XT1097)",290280,US,66.165.162.68,Message In A Bottle - Remastered 2003,The Police,Reggatta De Blanc,...,None,None,trackdone,trackdone,True,True,True,True,joseph-higaki,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94667,1db47bdf6a952082b0d013e44417908c,2024-12-04 06:55:39+00:00,2024,android,282653,ES,79.153.169.248,Voodoo,Godsmack,Godsmack,...,None,None,trackdone,trackdone,True,True,True,True,joseph-higaki,1.733295e+09
94668,a9efd464082ecb18990ff3c4143ecde6,2024-12-04 06:58:59+00:00,2024,android,199906,ES,79.153.169.248,Last Resort,Papa Roach,Infest,...,None,None,trackdone,trackdone,True,True,True,True,joseph-higaki,1.733295e+09
94669,d7f7990841af62ed57b41ce914936d96,2024-12-04 07:04:05+00:00,2024,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,...,None,None,trackdone,trackdone,True,True,True,True,joseph-higaki,1.733296e+09
94670,d83f376f1e232ec174353f6786a51bd4,2024-12-04 07:26:58+00:00,2024,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,...,None,None,trackdone,unknown,True,True,True,True,joseph-higaki,1.733296e+09


In [67]:
import pyarrow as pa
import pyarrow.parquet as pq

def store_to_partition(df):
    table = pa.Table.from_pandas(df)
    destination_full_path= "gs://spotify-insights-pipeline-data/stg_spotify_streams"
    print(destination_full_path)
    pq.write_to_dataset(
        table,
        root_path=destination_full_path,
        partition_cols=['username', 'streamed_year']
    )

In [68]:
store_to_partition(dedup_by_hash)

gs://spotify-insights-pipeline-data/stg_spotify_streams


In [69]:
raw_spotify_streams_fetched = pd.read_parquet("gs://spotify-insights-pipeline-data/stg_spotify_streams", engine="pyarrow")
raw_spotify_streams_fetched


,stream_id,streamed_at,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,...,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,offline_timestamp,username,streamed_year
0,d6b75fe3cc818d5b5d247228da525414,2014-12-23 00:37:58+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",300826,US,66.165.162.68,Smells Like Teen Spirit,Nirvana,Nirvana,spotify:track:4hy4fb5D1KL50b3sng9cjw,...,None,appload,trackdone,True,True,True,True,NaN,joseph-higaki,2014
1,baa6e5eea30ec3376c952bd60533c06e,2014-12-23 00:40:47+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",169133,US,66.165.162.68,Rape Me,Nirvana,Nirvana,spotify:track:47KVHb6cOVBZbmXQweE5p7,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph-higaki,2014
2,26bc6434510998232ebaa64f2494f9d6,2014-12-23 00:46:44+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",356080,US,66.165.162.68,Sweet Child O' Mine,Guns N' Roses,Appetite For Destruction,spotify:track:7o2CTH4ctstm8TNelqjb51,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph-higaki,2014
3,b1acc11f3a473570ee358f7d5256222a,2014-12-23 00:49:35+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",170800,US,66.165.162.68,Rock And Roll All Nite,KISS,Greatest Kiss,spotify:track:3qei6Em6zqxjFuPfhhghWM,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph-higaki,2014
4,d65acd45c2786b387df0ebfebe340eb1,2014-12-23 00:54:26+00:00,"Android OS 4.4.4 API 19 (motorola, XT1097)",290280,US,66.165.162.68,Message In A Bottle - Remastered 2003,The Police,Reggatta De Blanc,spotify:track:2inge47GpvkHhe0VCXP8Zd,...,None,trackdone,trackdone,True,True,True,True,NaN,joseph-higaki,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94667,1db47bdf6a952082b0d013e44417908c,2024-12-04 06:55:39+00:00,android,282653,ES,79.153.169.248,Voodoo,Godsmack,Godsmack,spotify:track:5uGsG0LfotfWDq6hql4h53,...,None,trackdone,trackdone,True,True,True,True,1.733295e+09,joseph-higaki,2024
94668,a9efd464082ecb18990ff3c4143ecde6,2024-12-04 06:58:59+00:00,android,199906,ES,79.153.169.248,Last Resort,Papa Roach,Infest,spotify:track:5W8YXBz9MTIDyrpYaCg2Ky,...,None,trackdone,trackdone,True,True,True,True,1.733295e+09,joseph-higaki,2024
94669,d7f7990841af62ed57b41ce914936d96,2024-12-04 07:04:05+00:00,android,304840,ES,79.153.169.248,Uprising,Muse,The Resistance,spotify:track:4VqPOruhp5EdPBeR92t6lQ,...,None,trackdone,trackdone,True,True,True,True,1.733296e+09,joseph-higaki,2024
94670,d83f376f1e232ec174353f6786a51bd4,2024-12-04 07:26:58+00:00,android,148181,ES,79.117.89.98,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,spotify:track:59WN2psjkt1tyaxjspN8fp,...,None,trackdone,unknown,True,True,True,True,1.733296e+09,joseph-higaki,2024
